In [ ]:
# https://sentinelhub-py.readthedocs.io/en/latest/configure.html#

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import rasterio as rs
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import os
import datetime

from PIL import Image
from sentinelhub import WebFeatureService, BBox, CRS, DataSource, WmsRequest, WcsRequest, MimeType, get_area_info
from IPython.display import clear_output
from rasterio.plot import reshape_as_image

palette = sns.color_palette("coolwarm", 5)
instance_id = ''

In [ ]:
df = pd.read_csv('../data/prediction_dataset.csv', dtype={'id':str})
print('gathered %s samples' % len(df))

df.head()

In [ ]:
 def plot_image(image, factor=1):
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)

coords_visuals_wgs84 = [20.303942, 51.420241, 20.311623, 51.425043]
bbox_visuals = BBox(bbox=coords_visuals_wgs84, crs=CRS.WGS84)

wms_visuals_request = WcsRequest(layer='FALLOWINITIAL',
                                bbox=bbox_visuals,
                                time='latest',
                                resx='10m', resy='10m',
                                maxcc=0.03,
                                instance_id=instance_id) 

wms_visuals = wms_visuals_request.get_data()
plot_image(wms_visuals[-1][:, :, 3])

In [ ]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    
    block = int(round(bar_length * progress)) 
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format("#" * block + "-" * (bar_length - block), progress * 100)
    print(text)    

In [ ]:
acquisition_time = ('2018-08-01', '2018-08-31')
maxcc = 0.03

directory_template = '../data/sentinel/predict/%s'

for i, (index, row) in enumerate(df.iterrows()):
    update_progress(i / len(df))

    data_folder = directory_template % row['id']
    
    if not os.path.exists(data_folder):  
        coords_wgs84 = [row['min_lng'], row['min_lat'], row['max_lng'], row['max_lat']]
            
        wms_fallow_request = WcsRequest(data_folder=data_folder,
                                        layer='FALLOWINITIAL',
                                        bbox=BBox(bbox=coords_wgs84, crs=CRS.WGS84),
                                        time=acquisition_time,
                                        resx='10m', resy='10m',
                                        maxcc=maxcc,
                                        time_difference=datetime.timedelta(days=31),
                                        image_format=MimeType.TIFF_d32f,
                                        instance_id=instance_id)

        wms_fallow_request.get_data(save_data=True, redownload=False)
            
update_progress(1)

In [ ]:
ids = []
files = []
dates = []

tiff_glob = '../data/sentinel/predict/*/*.tiff'

for f in glob.glob(tiff_glob): 
    files.append(os.path.basename(f))
    ids.append(os.path.basename(os.path.dirname(f)))
    dates.append(f.split('_')[7])
 
predict_data = pd.DataFrame({
    'id': ids,
    'file': files,
    'date': dates,
})  
predict_data.to_csv('../data/predict.csv')

predict_data.head()